Необходимые импорты

In [2]:
import pandas as pd

### **Задание 1**

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

-   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
-   для источников paid и email из России ставим ad;
-   для источников paid и email не из России ставим other;
-   все остальные варианты берём из traffic_source без изменений.


Загрузим датасет

In [9]:
df_log = pd.read_csv('./visit_log.csv', sep=';')

In [4]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       18938 non-null  int64 
 1   visit_id        18938 non-null  object
 2   url             18938 non-null  object
 3   region          18938 non-null  object
 4   user_id         18938 non-null  object
 5   traffic_source  18938 non-null  object
dtypes: int64(1), object(5)
memory usage: 887.8+ KB


In [5]:
df_log['region'].unique()

array(['Russia', 'China', 'India', 'Ukraine', 'Belarus', 'Australia'],
      dtype=object)

Создадим и заполним столбец **'data_source'** согласно правилам

In [23]:
%%time

df_log.loc[df_log.traffic_source == 'yandex', 'source_type'] = 'organic'
df_log.loc[df_log.traffic_source == 'google', 'source_type'] = 'organic'
df_log.loc[(df_log.traffic_source.isin(['paid', 'email'])) &
           (df_log['region'] == 'Russia'), 'source_type'] = 'ad'
df_log.loc[(df_log.traffic_source.isin(['paid', 'email'])) &
           (df_log.region != 'Russia'), 'source_type'] = 'other'
df_log.loc[df_log['source_type'].isna(), 'source_type'] = df_log['traffic_source']
df_log.sample(10)

CPU times: user 28.9 ms, sys: 7.84 ms, total: 36.7 ms
Wall time: 34.8 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
5739,1550015235,63a68ba89c,https://host.ru/fd32c3d2de55949865,Belarus,eb02c3917f,direct,direct
2953,1549998574,da93e11af4,https://host.ru/6fda01ec57f23abc9e,Belarus,fcefeed6eb,google,organic
6188,1550017927,dcaf55f610,https://host.ru/ac2d98a5a63e7c6e28,Russia,6dd3906cb0,direct,direct
11229,1550047817,270febf1a4,https://host.ru/c8d9213a31839f9a3a,Russia,67c68701d6,direct,direct
1251,1549988158,d286dd1fa5,https://host.ru/3c19b4ef7371864fa3,Russia,cd91a91068,yandex,organic
15673,1550074396,b6119bfefe,https://host.ru/3004a8273caeef2867,Belarus,6509a02703,email,other
6953,1550022420,a7aaf02acc,https://host.ru/39fa884393666d45fc,Belarus,0099f22c43,email,other
10328,1550042400,ee67aec391,https://host.ru/2fcb223266880ccf57,Russia,8db30b9e02,direct,direct
17828,1550087540,c7fd5493db,https://host.ru/c2382eb3d6afc8d0f3,Belarus,f7db4b25c0,paid,other
6576,1550020179,5c1d7e0c94,https://host.ru/8e88d4703848cc0ec4,Russia,eb99c5bbfd,paid,ad


### **Задание 2**
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

Загрузим датасет

In [24]:
df_url = pd.read_csv('./URLs.txt')

In [25]:
df_url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [26]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     93 non-null     object
dtypes: object(1)
memory usage: 876.0+ bytes


Отфильтруем новости по заданному шаблону, используя регулярное выражение

In [32]:
df_news_filtered = df_url.loc[df_url['url'].str.contains(r'/\d{8}-', regex=True)]
df_news_filtered.sample(10)

,url
53,/starlife/36140271-skandalnoe-foto-britni-spir...
47,/articles/36193368-donbass-reshil-udarit-pervym/
15,/video/36012692-tramp-otkrestilsya-ot-rossii-i...
16,/starlife/36174817-guzeeva-rasstalas-s-muzhem/
92,/world/36103095-dominikanskih-zhurnalistov-ubi...
14,/politics/35638742-nesolidno-mariya-zaharova-s...
28,/crazy-world/36193471-seks-bezhentsy-vernulis-...
69,/articles/36170146-rossiya-reshila-postavlyat-...
42,/video/36006794-pogibshaya-melamed-ostavila-pr...
80,/sport/36235520-v-britanii-razgorelsya-krupney...


### **Задание 3**
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

Загрузим датасет

In [70]:
df_ratings = pd.read_csv('./ratings.csv')
df_ratings.info()
df_ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Подсчитаем время жизни и отфильтруем пользователей согласно условиям

In [78]:
%%time

df_filtered = (
    df_ratings.groupby('userId').agg(
        rating_count=('rating', 'count'),
        lifetime=('timestamp', lambda x: max(x) - min(x))
    ).reset_index()
)

df_filtered = df_filtered[df_filtered['rating_count'] > 100]

df_filtered.head()

CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 120 ms


,userId,rating_count,lifetime
3,4,204,203560
7,8,116,85187
14,15,1700,471393496
16,17,363,8053
18,19,423,5282


### **Задание 4**
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
аналогичную таблицу по типам выручки с указанием адреса клиента.
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Сформируем датасеты и удалим дубликаты

In [125]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981, 666]
    }
)

rzd = rzd.groupby('client_id').sum().reset_index()

rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,1647


In [126]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98, 666]
    }
)

auto = auto.groupby('client_id').sum().reset_index()

auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,764


In [127]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118, 118],
        'air_revenue': [81, 4, 13, 173, 666]
    }
)

air = air.groupby('client_id').sum().reset_index()

air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,839


In [131]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118, 119],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4', 'Лизюкова 36']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4
8,119,Лизюкова 36


Объединим датасеты

In [132]:
df_summary = (
    rzd.merge(auto, on='client_id', how='outer')
    .merge(air,  on='client_id', how='outer')
).fillna(0)

df_summary

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,1647.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,764.0,13.0
7,118,0.0,0.0,839.0


Добавим адреса клиентов

In [137]:
df_summary_addr = df_summary.merge(client_base, on='client_id', how='outer').fillna(0)

df_summary_addr

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,1647.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,764.0,13.0,Панфиловская 33
7,118,0.0,0.0,839.0,Мастеркова 4
8,119,0.0,0.0,0.0,Лизюкова 36
